In [4]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.3. Summary Statistics/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.3. Summary Statistics/Output')
cur_date = "041322"

library(readxl)
library(tidyverse)
library(reshape2)
library(car)

#reading in files
feature_ranking = read_csv("041222_Significant_MFs_noise.csv")
demographics_df = data.frame(read_excel("SubjectInfo_022822.xlsx", sheet = 2))
molecular_feature_df = read_csv("20220217_mf_qrilc_w40.csv") 

Rows: 3877 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): MF
dbl (1): MeanDecreaseGini

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 59 Columns: 3873
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr    (1): sample
dbl (3872): MF_2, MF_3, MF_4, MF_5, MF_6, MF_7, MF_11, MF_12, MF_13, MF_14, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Generating a dataframe that includes:

1. MF Name
2. RF Importance Metrics
3. P. Value from t test
4. P Adj
5. Overall avg. abundance
6. Avg. abundance in Ecig users
7. Avg. abundance in NS
8. log 2(avg. fold change)

In [5]:
#changing colnames to combine
feature_ranking = feature_ranking %>%
    rename(Variable_Importance_Rank = MeanDecreaseGini)

head(feature_ranking)

MF,Variable_Importance_Rank
<chr>,<dbl>
MF_97,2.6118845
MF_5,1.8559144
MF_292,0.7185532
MF_7221,0.3767431
MF_29367,0.2427987
MF_6955,0.1876891


In [6]:
#calculating p values from t test
molecular_feature_df = molecular_feature_df %>%
    rename(Smoking_Status = sample) %>%
    mutate(Smoking_Status = ifelse(grepl("Ecig", Smoking_Status), "Ecig", "NS"))

head(molecular_feature_df)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [7]:
longer_mf_df_w_Ecig40 = molecular_feature_df %>%
    pivot_longer(cols = 2:3873, names_to = "MF", values_to = "Value")

head(longer_mf_df_w_Ecig40)

Smoking_Status,MF,Value
<chr>,<chr>,<dbl>
NS,MF_2,19.86186
NS,MF_3,17.86943
NS,MF_4,19.21394
NS,MF_5,18.02809
NS,MF_6,17.46636
NS,MF_7,18.34174


In [8]:
t_test_values = function(df, p_value_name, padjust_name){
    
    single_MF = unique(longer_mf_df_w_Ecig40$MF)
    
    p_values_df = data.frame()
    #iterating through each MF
    for (i in 1:length(single_MF)){
        #NS df
        NS_df = df %>%
            filter(Smoking_Status == "NS", MF == single_MF[i])
        #Ecig df
        Ecig_df = df %>%
            filter(Smoking_Status == "Ecig", MF == single_MF[i])

        #t test
        t_test = t.test(NS_df$Value, Ecig_df$Value)
        pvalue = t_test$p.value
        
        #saving values to df
        p_values_df = rbind(p_values_df, cbind(single_MF[i], pvalue))
    }
    
    #adding col names
    colnames(p_values_df) = c("MF", p_value_name)
    #calculating padj values
    p_values_df[[padjust_name]] = p.adjust(as.numeric(as.character(p_values_df$`P Value`)), method = "fdr")
    
    return(p_values_df)
}

#calling fn
pvalue_w_Ecig_40 = t_test_values(longer_mf_df_w_Ecig40, "P Value", "P Adjust")

head(pvalue_w_Ecig_40)

,MF,P Value,P Adjust
,<chr>,<chr>,<dbl>
1,MF_2,0.157890632296302,6.969766e-01
2,MF_3,0.183088265453325,7.122719e-01
3,MF_4,0.396463535957884,8.333091e-01
4,MF_5,3.67620776654923e-08,7.117138e-05
5,MF_6,0.901198651169926,9.794800e-01
6,MF_7,0.102708109504866,6.352808e-01


In [9]:
#also calculating p adj using covariates using ANCOVA
#first making a combined df w/ covariate and MF data ready for ANCOVA
covariates_MF_df = molecular_feature_df_w_Ecig_40 %>%
    pivot_longer(cols = 2:3873, names_to = "MF", values_to = "Value") %>%
    rename(SubjectID = sample) %>%
    inner_join(demographics_df) %>%
    select(-Original_SubjectID) %>%
    #dichotomizing race to prevent from having small groups
    mutate(Race = ifelse(Race != "W", "NW", "W")) %>%
    #scaling within each MF
    group_by(MF) %>%
    mutate(Scaled_Value = scale(Value))

#making categorical variables into factors
covariates_MF_df$Sex = factor(covariates_MF_df$Sex)
covariates_MF_df$Race = factor(covariates_MF_df$Race)

head(covariates_MF_df)

Joining, by = "SubjectID"


SubjectID,MF,Value,Exposure,Sex,Race,Age,BMI,Scaled_Value
<chr>,<chr>,<dbl>,<chr>,<fct>,<fct>,<dbl>,<dbl>,"<dbl[,1]>"
NS_070,MF_2,19.86186,NS,F,W,33,32.3,-0.9275484
NS_070,MF_3,17.86943,NS,F,W,33,32.3,-1.5269284
NS_070,MF_4,19.21394,NS,F,W,33,32.3,-1.3048726
NS_070,MF_5,18.02809,NS,F,W,33,32.3,-0.5192701
NS_070,MF_6,17.46636,NS,F,W,33,32.3,-0.6704138
NS_070,MF_7,18.34174,NS,F,W,33,32.3,-1.7115245


In [10]:
#now running ancova
get_ancova = function(scaled_df){
  
    #getting unique MFs to iterate through
    single_MF = unique(longer_mf_df_w_Ecig40$MF)
    
    values_df = data.frame()
    for (i in 1:length(single_MF)){
        #the covariate must go first for Type 1, but we want type III
        ancova = aov(Scaled_Value ~ Sex + Race + Age + BMI + Exposure, data = scaled_df %>%
                        filter(MF == single_MF[i])) 

        anova_type3 = Anova(ancova, type = 'III')
        summary = anova_type3[2,3:4]

        #saving values
        saved_values = cbind(single_MF[i], summary$`Pr(>F)`)
        values_df = rbind(values_df, saved_values)
    }
    
    #assigning col names
    colnames(values_df) = c("MF", "P_Adj_covariates")              
    return(values_df)
}

#calling fn
covariate_adjusted_pvalues = get_ancova(covariates_MF_df)

In [11]:
head(covariate_adjusted_pvalues)

,MF,P_Adj_covariates
,<chr>,<chr>
1,MF_2,0.62890819576864
2,MF_3,0.26342648788262
3,MF_4,0.226677831790364
4,MF_5,0.661156092545463
5,MF_6,0.949103199436608
6,MF_7,0.91060564555964


In [12]:
#creating p value df
pvalue_df = inner_join(pvalue_w_Ecig_40, covariate_adjusted_pvalues)
head(pvalue_df)

Joining, by = "MF"


,MF,P Value,P Adjust,P_Adj_covariates
,<chr>,<chr>,<dbl>,<chr>
1,MF_2,0.157890632296302,6.969766e-01,0.62890819576864
2,MF_3,0.183088265453325,7.122719e-01,0.26342648788262
3,MF_4,0.396463535957884,8.333091e-01,0.226677831790364
4,MF_5,3.67620776654923e-08,7.117138e-05,0.661156092545463
5,MF_6,0.901198651169926,9.794800e-01,0.949103199436608
6,MF_7,0.102708109504866,6.352808e-01,0.91060564555964


In [13]:
#calculating overall avg. abundance and within smoking groups
avg_abundance_w_Ecig_40 = longer_mf_df_w_Ecig40 %>%
    group_by(MF) %>%
    summarize(Avg_Abundance = mean(Value))

#stratified by smoking group
avg_abundance_w_Ecig_40_stratified = longer_mf_df_w_Ecig40 %>%
    group_by(Smoking_Status, MF) %>%
    summarize(Avg_Abundance_stratified = mean(Value))

head(avg_abundance_w_Ecig_40)
head(avg_abundance_w_Ecig_40_stratified)

`summarise()` has grouped output by 'Smoking_Status'. You can override using
the `.groups` argument.


MF,Avg_Abundance
<chr>,<dbl>
MF_100,17.90073
MF_1000,20.47764
MF_10009,19.32521
MF_1001,19.65870
MF_10019,18.51077
MF_1002,21.29595


Smoking_Status,MF,Avg_Abundance_stratified
<chr>,<chr>,<dbl>
Ecig,MF_100,18.25741
Ecig,MF_1000,20.44849
Ecig,MF_10009,19.53787
Ecig,MF_1001,19.63929
Ecig,MF_10019,18.69506
Ecig,MF_1002,21.54147


In [14]:
#changing the stratified dfs to making combining easier
#first splitting dfs
split_avg_abundance_w_Ecig_40_stratified = avg_abundance_w_Ecig_40_stratified %>%
    group_by(Smoking_Status) %>%
    group_split()

#adding smoking status to the title
Ecig_avg_abundance = split_avg_abundance_w_Ecig_40_stratified[[1]] %>%
    select(-Smoking_Status) %>%
    rename(Ecig_Avg_Abundance = Avg_Abundance_stratified)
NS_avg_abundance = split_avg_abundance_w_Ecig_40_stratified[[2]] %>%
    select(-Smoking_Status) %>%
    rename(NS_Avg_Abundance = Avg_Abundance_stratified)

head(Ecig_avg_abundance)

MF,Ecig_Avg_Abundance
<chr>,<dbl>
MF_100,18.25741
MF_1000,20.44849
MF_10009,19.53787
MF_1001,19.63929
MF_10019,18.69506
MF_1002,21.54147


In [15]:
avg_abundance_df = cbind(avg_abundance_w_Ecig_40, inner_join(Ecig_avg_abundance, NS_avg_abundance)[,2:3])
head(avg_abundance_df)

Joining, by = "MF"


,MF,Avg_Abundance,Ecig_Avg_Abundance,NS_Avg_Abundance
,<chr>,<dbl>,<dbl>,<dbl>
1,MF_100,17.90073,18.25741,17.47800
2,MF_1000,20.47764,20.44849,20.51219
3,MF_10009,19.32521,19.53787,19.07316
4,MF_1001,19.65870,19.63929,19.68171
5,MF_10019,18.51077,18.69506,18.29235
6,MF_1002,21.29595,21.54147,21.00496


In [16]:
#calculating log 2 fold change
#FC = avg. abundance across ecig users/ avg. abundance across NS
log2FC = function(abundance_df, Ecig_col, NS_col, log2name){
    
    single_MF = unique(abundance_df$MF)
    
    values_df = data.frame()
    #iterating through each MF
    for (i in 1:length(single_MF)){
    filtered_df = abundance_df %>%
        filter(MF == single_MF[i])
    
    log2_FC_value = log2(mean(filtered_df[[Ecig_col]])/mean(filtered_df[[NS_col]]))
   
    #saving values
    values_df = rbind(values_df, cbind(single_MF[i], log2_FC_value))
    }
    
    #adding col names
    colnames(values_df) = c("MF", log2name)
    return(values_df)
    
}


#calling fn
log2FC_w_Ecig_40 = log2FC(avg_abundance_df, "Ecig_Avg_Abundance", "NS_Avg_Abundance", "log2FC")
head(log2FC_w_Ecig_40)

,MF,log2FC
,<chr>,<chr>
1,MF_100,0.0629415491221518
2,MF_1000,-0.00448705289641026
3,MF_10009,0.0347293327160267
4,MF_1001,-0.00311310352076818
5,MF_10019,0.0314169810277366
6,MF_1002,0.036386465819493


In [17]:
#combining into 1 df
combined_df = inner_join(inner_join(feature_ranking, pvalue_df), inner_join(avg_abundance_df, log2FC_w_Ecig_40)) 
#combined_df[,2:15] = apply(combined_df[,2:15], 2, as.numeric)
head(combined_df)

Joining, by = "MF"
Joining, by = "MF"
Joining, by = "MF"


MF,Variable_Importance_Rank,P Value,P Adjust,P_Adj_covariates,Avg_Abundance,Ecig_Avg_Abundance,NS_Avg_Abundance,log2FC
<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
MF_97,2.6118845,4.85697347107151e-11,1.880620e-07,0.611414602623618,19.34139,21.21361,17.12246,0.30910031499844
MF_5,1.8559144,3.67620776654923e-08,7.117138e-05,0.661156092545463,19.85841,21.94706,17.38297,0.336352734212379
MF_292,0.7185532,6.16003424173649e-05,7.950551e-02,0.884525092981179,19.82281,21.43326,17.91412,0.258754537758257
MF_7221,0.3767431,0.00150616004733428,1.715251e-01,0.585310040066992,17.45214,18.00508,16.79680,0.100218242991128
MF_29367,0.2427987,0.0356632079780345,4.568437e-01,0.883850458066319,18.17888,18.53933,17.75168,0.0626335642359544
MF_6955,0.1876891,0.000143994315918898,8.868030e-02,0.901880741900407,18.79478,19.61510,17.82254,0.138261642397508


In [18]:
#exporting table 
write.csv(combined_df, paste0(Output,"/", cur_date, "_MF_Expression_Table.csv"), row.names = FALSE)